#### Librerias Necesarias

In [1]:
(require '[clojure.string :as s])
(defn toInt[x] (Integer/parseInt x));Java
(defn toDouble[x] (Double/parseDouble x));Java

#'user/toDouble

Clase de Pelicula

In [2]:
(defrecord Movie [id title year age imdb rottenTomato netflix hulu primeVideo disneyPlus  director geners country lenguage runtime]
    Object 
    (toString [this]
              (format "%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s" id title year age imdb rottenTomato netflix hulu primeVideo disneyPlus director geners country lenguage runtime)
              )
    )

user.Movie

Lectura de informacion

In [3]:
(def data (vec (rest (s/split-lines (slurp "MoviesOnStreamingPlatforms_updated.csv") ) ) ) )

#'user/data

Conversion de datos

In [4]:
(defn toMovie[ record ]
    (Movie. 
     (toInt(get record 0))  
     (get record 1)  
     (toInt (get record 2)) 
     (get record 3) (toDouble (get record 4)) (toDouble (get record 5)) 
                                (toInt (get record 6)) 
                                (toInt (get record 7))
                                (toInt (get record 8))
                                (toInt (get record 9))
                                (get record 10)
                                (get record 11)
                                (get record 12)
                                (get record 13)
                                (toInt (get record 14)) 
                               ))

#'user/toMovie

In [5]:
(defn get-Movies-lazy [datos] 
    (lazy-seq (cons (toMovie (s/split (first datos) #","))(get-Movies-lazy (rest datos))) )
    )

#'user/get-Movies-lazy

In [6]:
(defn get-Vector-Valor [datos, valor]
    (lazy-seq (cons (valor (first datos))(get-Vector-Valor (rest datos) valor)))
    )

#'user/get-Vector-Valor

In [7]:
(def movies (take 20 (get-Movies-lazy data)))

#'user/movies

In [8]:
(def imdb (vec (take 20 (get-Vector-Valor movies :imdb))))

#'user/imdb

In [9]:
(def rottonTomato (vec (take 20 (get-Vector-Valor movies :rottonTomato))))

#'user/rottonTomato

In [10]:
(def runtime (vec (take 20 (get-Vector-Valor movies :runtime))))

#'user/runtime

In [11]:
(def x0 (vec (take 20 (repeat 1))))

#'user/x0

In [12]:
(def matrixB [x0 imdb runtime])

#'user/matrixB

Transpuesta

In [13]:
(defn get-New-Col [matrix]
    (def newCol [(first (get matrix 0))(first (get matrix 1)) (first (get matrix 2))])
    newCol
    )

#'user/get-New-Col

In [14]:
(defn transpuesta [matrix]
    (loop [mResult (atom '()) mTemp matrix] 
         (if (= 0 (count (get mTemp 1)))
             mResult
             (do
                 (def m (get-New-Col mTemp))
                 (swap! mResult conj m)
                 (recur mResult [(vec (rest (get mTemp 0)))(vec (rest (get mTemp 1))) (vec (rest (get mTemp 2)))])
                 )
             )
        )
    )

#'user/transpuesta

In [15]:
(def matrixA (vec @(transpuesta matrixB)))

#'user/matrixA

Multiplicador de matrices


In [16]:
matrixA

[[1 8.9 91] [1 8.9 87] [1 8.9 130] [1 8.9 50] [1 8.9 89] [1 8.9 102] [1 8.9 75] [1 8.9 135] [1 9.0 90] [1 9.0 60] [1 9.0 152] [1 9.1 105] [1 9.1 56] [1 9.1 166] [1 9.3 132] [1 9.3 68] [1 9.3 60] [1 9.3 83] [1 9.3 90] [1 9.3 61]]

In [17]:
 matrixB

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1] [9.3 9.3 9.3 9.3 9.3 9.3 9.1 9.1 9.1 9.0 9.0 9.0 8.9 8.9 8.9 8.9 8.9 8.9 8.9 8.9] [61 90 83 60 68 132 166 56 105 152 60 90 135 75 102 89 50 130 87 91]]

In [18]:
(defn get-Mult-Val [matrixA matrixB]
   
    (def newVal 
        (+ 
         (* (get matrixA 0)(get matrixB 0))
         (* (get matrixA 1)(get matrixB 1))
         (* (get matrixA 2)(get matrixB 2))
         ))
  
    newVal
)

#'user/get-Mult-Val

In [19]:
(defn multiplicacion [matrixA matrixB]
    (loop [mResult (atom '())  n 0]
         (if (= n (count matrixA))
             mResult
             (do
                 (def vectorTemp [(get-Mult-Val (get matrixB 0) (get matrixA n)) (get-Mult-Val (get matrixB 1) (get matrixA n)) (get-Mult-Val (get matrixB 2) (get matrixA n))])
                 (println vectorTemp)
                 (swap! mResult conj vectorTemp)
                 (recur mResult (inc n))
                 
             )
        )
    )
)

#'user/multiplicacion

In [20]:
(reverse @(multiplicacion matrixA matrixB))

[100.9 938.3700000000001 8415.0]
[96.9 901.1700000000001 8083.0]
[139.9 1301.07 11652.0]
[59.9 557.07 5012.0]
[98.9 919.7700000000001 8249.0]
[111.9 1040.67 9328.0]
[84.9 789.57 7087.0]
[144.9 1347.57 12067.0]
[100.0 930.0000000000001 8341.0]
[70.0 651.0 5851.0]
[162.0 1506.6000000000001 13487.0]
[115.1 1070.43 9595.0]
[66.1 614.73 5528.0]
[176.1 1637.7300000000002 14658.0]
[142.3 1323.39 11854.0]
[78.3 728.19 6542.0]
[70.3 653.79 5878.0]
[93.3 867.69 7787.0]
[100.3 932.7900000000001 8368.0]
[71.3 663.09 5961.0]


([100.9 938.3700000000001 8415.0] [96.9 901.1700000000001 8083.0] [139.9 1301.07 11652.0] [59.9 557.07 5012.0] [98.9 919.7700000000001 8249.0] [111.9 1040.67 9328.0] [84.9 789.57 7087.0] [144.9 1347.57 12067.0] [100.0 930.0000000000001 8341.0] [70.0 651.0 5851.0] [162.0 1506.6000000000001 13487.0] [115.1 1070.43 9595.0] [66.1 614.73 5528.0] [176.1 1637.7300000000002 14658.0] [142.3 1323.39 11854.0] [78.3 728.19 6542.0] [70.3 653.79 5878.0] [93.3 867.69 7787.0] [100.3 932.7900000000001 8368.0] [71.3 663.09 5961.0])